<a href="https://colab.research.google.com/github/MahimaGaikwad/Genetic-algorithm-to-find-optimize-model/blob/master/Finding_best_individual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pip install deap

     |████████████████████████████████| 163kB 4.5MB/s 


In [4]:
!pip install bitstring

In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split

from keras.layers import LSTM, Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

np.random.seed(1120)

In [23]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Global Energy Forecasting Competition 2012 - Wind Forecasting/train.csv')
data = np.reshape(np.array(data['wp1']),(len(data['wp1']),1))

train_data = data[0:17257]
test_data = data[17257:]

print(train_data.shape)
print(test_data.shape)

(17257, 1)
(1500, 1)


In [0]:
def prepare_dataset(data, window_size):
    X, Y = np.empty((0,window_size)), np.empty((0))
    for i in range(len(data)-window_size-1):
        X = np.vstack([X,data[i:(i + window_size),0]])
        Y = np.append(Y,data[i + window_size,0])   
    X = np.reshape(X,(len(X),window_size,1))
    Y = np.reshape(Y,(len(Y),1))
    return X, Y

In [0]:
def train_evaluate(ga_individual_solution):   
    # Decode genetic algorithm solution to integer for window_size and num_units
    window_size_bits = BitArray(ga_individual_solution[0:6])
    num_units_bits = BitArray(ga_individual_solution[6:]) 
    window_size = window_size_bits.uint
    num_units = num_units_bits.uint
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_units)
    
    # Return fitness score of 100 if window_size or num_unit is zero
    if window_size == 0 or num_units == 0:
        return 100, 
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    X,Y = prepare_dataset(train_data,window_size)
    X_train, X_val, y_train, y_val = split(X, Y, test_size = 0.20, random_state = 1120)
    
    # Train LSTM model and predict on validation set
    inputs = Input(shape=(window_size,1))
    x = LSTM(num_units, input_shape=(window_size,1))(inputs)
    predictions = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='adam',loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
    y_pred = model.predict(X_val)

    #print(X_train.shape, y_train.shape)
    #print(X_val.shape, y_val.shape)
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    
    return rmse,



In [13]:
population_size = 4
num_generations = 4
gene_length = 10

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)


Window Size:  36 , Num of Units:  2
Epoch 1/5
13776/13776 [==============================] - 13s 942us/step - loss: 0.0187
Epoch 2/5
13776/13776 [==============================] - 12s 888us/step - loss: 0.0087
Epoch 3/5
13776/13776 [==============================] - 13s 912us/step - loss: 0.0066
Epoch 4/5
13776/13776 [==============================] - 12s 894us/step - loss: 0.0059
Epoch 5/5
13776/13776 [==============================] - 12s 905us/step - loss: 0.0057
Validation RMSE:  0.07613214069942839 


Window Size:  56 , Num of Units:  8
Epoch 1/5
13760/13760 [==============================] - 21s 2ms/step - loss: 0.0128
Epoch 2/5
13760/13760 [==============================] - 22s 2ms/step - loss: 0.0063
Epoch 3/5
13760/13760 [==============================] - 21s 2ms/step - loss: 0.0058
Epoch 4/5
13760/13760 [==============================] - 22s 2ms/step - loss: 0.0058
Epoch 5/5
13760/13760 [==============================] - 21s 2ms/step - loss: 0.0057
Validation RMSE:  0.074282

In [14]:
best_individuals = tools.selBest(population,k = 1)
best_window_size = None
best_num_units = None

for bi in best_individuals:
    window_size_bits = BitArray(bi[0:6])
    num_units_bits = BitArray(bi[6:]) 
    best_window_size = window_size_bits.uint
    best_num_units = num_units_bits.uint
    print('\nWindow Size: ', best_window_size, ', Num of Units: ', best_num_units)


Window Size:  60 , Num of Units:  9


In [24]:
print(window_size_bits)

0b111100


In [25]:
print(num_units_bits)

0x9


In [15]:
X_train,y_train = prepare_dataset(train_data,best_window_size)
X_test, y_test = prepare_dataset(test_data,best_window_size)

inputs = Input(shape=(best_window_size,1))
x = LSTM(best_num_units, input_shape=(best_window_size,1))(inputs)
predictions = Dense(1, activation='linear')(x)
model = Model(inputs = inputs, outputs = predictions)
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)

Epoch 1/5
17196/17196 [==============================] - 29s 2ms/step - loss: 0.0188
Epoch 2/5
17196/17196 [==============================] - 29s 2ms/step - loss: 0.0061
Epoch 3/5
17196/17196 [==============================] - 30s 2ms/step - loss: 0.0058
Epoch 4/5
17196/17196 [==============================] - 29s 2ms/step - loss: 0.0057
Epoch 5/5
17196/17196 [==============================] - 28s 2ms/step - loss: 0.0057
Test RMSE:  0.09234263847427145


In [16]:
from keras.models import load_model
model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/Best_individual_model.json", "w") as json_file:
  json_file.write(model_json)
model.save("/content/gdrive/My Drive/Colab Notebooks/Best_individual_model.h5")
print("Savedmodel to drive")

Savedmodel to drive


In [17]:
print(X_train)

[[[0.045]
  [0.085]
  [0.02 ]
  ...
  [0.025]
  [0.04 ]
  [0.1  ]]

 [[0.085]
  [0.02 ]
  [0.06 ]
  ...
  [0.04 ]
  [0.1  ]
  [0.175]]

 [[0.02 ]
  [0.06 ]
  [0.045]
  ...
  [0.1  ]
  [0.175]
  [0.13 ]]

 ...

 [[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.09 ]
  [0.11 ]
  [0.11 ]]

 [[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.11 ]
  [0.11 ]
  [0.095]]

 [[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.11 ]
  [0.095]
  [0.14 ]]]


In [18]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(17196, 60, 1) (17196, 1)
(1439, 60, 1) (1439, 1)
